# 0.0 Imports

In [4]:
import pandas as pd 
import numpy as np 
import inflection
import re
import umap as umap

from sklearn import cluster as c
from sklearn import metrics 
from sklearn import ensemble as en

import s3fs
import fs

import sqlite3
from sqlalchemy import create_engine, text

## 0.2 Load Data

In [5]:
# path_s3 = 's3://s3-mentoria-deploy'
data = pd.read_excel( 's3://menotria-cluster-s3/Online Retail.xlsx' )

# 1.0 Data Description

In [6]:
df1 = data.copy()

## 1.1 Rename Columns

In [7]:
cols_old = ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country']
snakecase = lambda x: inflection.underscore( x ) # function to snakecase
cols_news = list( map( snakecase, cols_old ))
df1.columns = cols_news

## 1.3 Check NA

## 1.4 Replace NA

In [8]:
# remove na
df1 = df1.dropna(subset=['description', 'customer_id'])
df_missing = df1[df1['customer_id'].isna()]
print(f'Missing: {df_missing.shape[0]}')
print(f'Not Missing: {df1.shape[0]}')

Missing: 0
Not Missing: 406829


In [9]:
# replace Na
# create reference
backup = pd.DataFrame(df_missing['invoice_no'].drop_duplicates())
backup['customer_id'] = np.arange(19000, 19000+len(backup), 1)

# merge original data frame
df1 = pd.merge(df1, backup, on='invoice_no', how='left')

#coalesce
df1['customer_id'] = df1['customer_id_x'].combine_first(df1['customer_id_y'])

# drop columns
df1 = df1.drop(columns=['customer_id_x', 'customer_id_y'], axis=1)

## 1.5 Change dtypes

In [10]:
# invoice date
df1['invoice_date'] = pd.to_datetime( df1['invoice_date'], format='%d-%b-%y' )

# customer id
df1['customer_id'] = df1['customer_id'].astype( int )

# 2.0 Data Filtering

In [11]:
df2 = df1.copy()

In [12]:
# === Numerical attributes ====
df2_purchases = df2.loc[df2['unit_price'] >= 0.04, :]

# === Categorical attributes ====
df2 = df2[~df2['stock_code'].isin( ['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY', 'DCGSSGIRL','PADS', 'B', 'CRUK'] )]

# description
df2 = df2.drop( columns='description', axis=1 )

# map 
df2 = df2[~df2['country'].isin( ['European Community', 'Unspecified' ] ) ]

# quantity
df2_returns = df2.loc[df1['quantity'] < 0, :]
df2_purchases = df2.loc[df1['quantity'] >= 0, :]

# 3.0 Feature Engineering

In [13]:
df3 = df2_purchases.copy()

## 3.1 Feature Creation

In [14]:
# DF reference
df_ref = df3.drop( ['invoice_no', 'stock_code', 
                    'quantity', 'invoice_date', 'unit_price', 
                    'country'], axis=1 ).drop_duplicates( ignore_index=True )

### Gross Revenue

In [15]:
# Gross revenue
df2_purchases['gross_revenue'] = df2_purchases['quantity'] * df2_purchases['unit_price']

# Monetary
df_monetary = df2_purchases[['customer_id', 'gross_revenue']].groupby('customer_id').sum().reset_index()
df_ref = pd.merge(df_ref, df_monetary, on='customer_id', how='left')

/tmp/ipykernel_14013/1522764053.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_purchases['gross_revenue'] = df2_purchases['quantity'] * df2_purchases['unit_price']


### Recency - Days since the last purchase

In [16]:
# Recency - Last day purchase
# **inverter função**
df_recency = df2_purchases[['customer_id', 'invoice_date']].groupby( 'customer_id' ).max().reset_index()
df_recency['recency_days'] = ( df2_purchases['invoice_date'].max() - df_recency['invoice_date'] ).dt.days
df_recency = df_recency[['customer_id', 'recency_days']].copy()
# df_recency['recency_days'] = df_recency['recency_days'].apply( lambda x: 1 / x if x != 0 else 0)
df_ref = pd.merge( df_ref, df_recency, on='customer_id', how='left' )

### Quantity of purchases

In [17]:
# Numero de produtos
df_freq = (df2_purchases.loc[:, ['customer_id', 'invoice_no']].drop_duplicates()
                                                             .groupby( 'customer_id' )
                                                             .count()
                                                             .reset_index()
                                                             .rename( columns={'invoice_no': 'qtde_invoices'}) )
df_ref = pd.merge( df_ref, df_freq, on='customer_id', how='left' )
df_ref.isna().sum()

customer_id      0
gross_revenue    0
recency_days     0
qtde_invoices    0
dtype: int64

### Total of items purchased

In [18]:
# Numero de produtos
df_freq = (df2_purchases.loc[:, ['customer_id', 'quantity']].groupby( 'customer_id' ).sum()
                                                           .reset_index()
                                                           .rename( columns={'quantity': 'qtde_items'} ) )
df_ref = pd.merge( df_ref, df_freq, on='customer_id', how='left' )
df_ref.isna().sum()

customer_id      0
gross_revenue    0
recency_days     0
qtde_invoices    0
qtde_items       0
dtype: int64

### Totoal of unique products purchased

In [19]:
# Numero de produtos
df_freq = (df2_purchases.loc[:, ['customer_id', 'stock_code']].groupby( 'customer_id' ).count()
                                                           .reset_index()
                                                           .rename( columns={'stock_code': 'qtde_products'} ) )
df_ref = pd.merge( df_ref, df_freq, on='customer_id', how='left' )
df_ref.isna().sum()

customer_id      0
gross_revenue    0
recency_days     0
qtde_invoices    0
qtde_items       0
qtde_products    0
dtype: int64

### Avg Ticket

In [20]:
# Avg Ticket
df_avg_ticket = df2_purchases.loc[:, ['customer_id', 'gross_revenue']].groupby( 'customer_id' ).mean().reset_index().rename( columns={'gross_revenue':'avg_ticket'} )
df_ref = pd.merge( df_ref, df_avg_ticket, on='customer_id', how='left')
df_ref.isna().sum()

customer_id      0
gross_revenue    0
recency_days     0
qtde_invoices    0
qtde_items       0
qtde_products    0
avg_ticket       0
dtype: int64

### Avg recency days

In [21]:
# Average recency days
df_aux = df2[['customer_id', 'invoice_date']].drop_duplicates().sort_values( ['customer_id', 'invoice_date'], ascending=False )
df_aux['next_customer_id'] = df_aux['customer_id'].shift() # next customer
df_aux['previous_date'] = df_aux['invoice_date'].shift() # next invoince date

df_aux['avg_recency_days'] = df_aux.apply( lambda x: ( x['invoice_date'] - x['previous_date'] ).days if x['customer_id'] == x['next_customer_id'] else 0, axis=1 )
# df_aux['avg_recency_days'] = df_aux['avg_recency_days'].apply( lambda x: 1/x if x !=0 else 0)

df_aux = df_aux.drop( ['invoice_date', 'next_customer_id', 'previous_date'], axis=1 ).dropna()

# average recency 
df_avg_recency_days = df_aux.groupby( 'customer_id' ).mean().reset_index()

# merge
df_ref = pd.merge( df_ref, df_avg_recency_days, on='customer_id', how='left' )
df_ref.isna().sum()

customer_id         0
gross_revenue       0
recency_days        0
qtde_invoices       0
qtde_items          0
qtde_products       0
avg_ticket          0
avg_recency_days    0
dtype: int64

### Frequency of purchases
- frequency = event / time 

In [22]:
df_aux = ( df2_purchases[['customer_id', 'invoice_no', 'invoice_date']].drop_duplicates()
                                                             .groupby( 'customer_id')
                                                             .agg( max_ = ( 'invoice_date', 'max' ), 
                                                                   min_ = ( 'invoice_date', 'min' ),
                                                                   days_= ( 'invoice_date', lambda x: ( ( x.max() - x.min() ).days ) + 1 ),
                                                                   buy_ = ( 'invoice_no', 'count' ) ) ).reset_index()
# Frequency
df_aux['frequency'] = df_aux[['buy_', 'days_']].apply( lambda x: x['buy_'] / x['days_'] if  x['days_'] != 0 else 0, axis=1 )

# Merge
df_ref = pd.merge( df_ref, df_aux[['customer_id', 'frequency']], on='customer_id', how='left' )

df_ref.isna().sum()

customer_id         0
gross_revenue       0
recency_days        0
qtde_invoices       0
qtde_items          0
qtde_products       0
avg_ticket          0
avg_recency_days    0
frequency           0
dtype: int64

### Number of returns

In [23]:
# Number of Returns
df_returns = df2_returns[['customer_id', 'quantity']].groupby( 'customer_id' ).sum().reset_index().rename( columns={'quantity':'qtde_returns'} )
df_returns['qtde_returns'] = df_returns['qtde_returns'] * -1

df_ref = pd.merge( df_ref, df_returns, how='left', on='customer_id' )
df_ref.loc[df_ref['qtde_returns'].isna(), 'qtde_returns'] = 0

df_ref.isna().sum()

customer_id         0
gross_revenue       0
recency_days        0
qtde_invoices       0
qtde_items          0
qtde_products       0
avg_ticket          0
avg_recency_days    0
frequency           0
qtde_returns        0
dtype: int64

### Basket Size

In [24]:
df_aux = ( df2_purchases.loc[:, ['customer_id', 'invoice_no', 'quantity']].groupby( 'customer_id' )
                                                                            .agg( n_purchase=( 'invoice_no', 'nunique'),
                                                                                  n_products=( 'quantity', 'sum' ) )
                                                                            .reset_index() )

# calculation
df_aux['avg_basket_size'] = df_aux['n_products'] / df_aux['n_purchase']

# merge
df_ref = pd.merge( df_ref, df_aux[['customer_id', 'avg_basket_size']], how='left', on='customer_id' )
df_ref.isna().sum()

customer_id         0
gross_revenue       0
recency_days        0
qtde_invoices       0
qtde_items          0
qtde_products       0
avg_ticket          0
avg_recency_days    0
frequency           0
qtde_returns        0
avg_basket_size     0
dtype: int64

### Unique Basket Size

In [25]:
df_aux = ( df2_purchases.loc[:, ['customer_id', 'invoice_no', 'stock_code']].groupby( 'customer_id' )
                                                                            .agg( n_purchase=( 'invoice_no', 'nunique'),
                                                                                   n_products=( 'stock_code', 'nunique' ) )
                                                                            .reset_index() )

# calculation
df_aux['avg_unique_basket_size'] = df_aux['n_products'] / df_aux['n_purchase']

# merge
df_ref = pd.merge( df_ref, df_aux[['customer_id', 'avg_unique_basket_size']], how='left', on='customer_id' )
df_ref.isna().sum()

customer_id               0
gross_revenue             0
recency_days              0
qtde_invoices             0
qtde_items                0
qtde_products             0
avg_ticket                0
avg_recency_days          0
frequency                 0
qtde_returns              0
avg_basket_size           0
avg_unique_basket_size    0
dtype: int64

# 4.0 EDA

In [26]:
df4 = df_ref.copy()
df4 = df4.dropna()

## 4.3 Estudo de espaços

In [27]:
df43 = df4.drop(columns=['customer_id'], axis=1).copy()

In [28]:
X = df43.copy()

### 4.3.4 Tree-based Embedding

In [29]:
X = df43.drop(columns=['gross_revenue'], axis=1).copy()
y = df43['gross_revenue'].copy()

# model definiton
forest = en.RandomForestRegressor( n_estimators=100, 
                                  random_state=42,
                                   criterion='friedman_mse' )

# model training
forest.fit( X, y )

# predict
df_leaf = pd.DataFrame(forest.apply( X ))

In [30]:
reducer = umap.UMAP( n_neighbors=80, random_state=42 )
embedding = reducer.fit_transform( df_leaf )

# embedding
df_tree = pd.DataFrame()
df_tree['embedding_x'] = embedding[:, 0]
df_tree['embedding_y'] = embedding[:, 1]

/home/natan/miniconda3/envs/mentodeploy/lib/python3.10/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


# 5.0 Data Preparation

In [31]:
df5 = df4.copy()

# 6.0 Feature Selection

In [32]:
df6 = df5.copy()

# 7.0 Hyperparameter Fine-Tunning

# 8.0 Model Training

## 8.1 K-Means

In [33]:
X = df_tree.copy()
# Model definition
k = 10
kmeans = c.KMeans( init='random', n_clusters=k, n_init=10, random_state=42, max_iter=300 )

# model training
kmeans.fit( X )

# clustering
labels = kmeans.labels_

## 8.2 Cluster Validation

In [34]:
# WSS (Within-cluster sum of square)
print(kmeans.inertia_)

# SS (Silhouette Score)
print(metrics.silhouette_score( X, labels, metric='euclidean'))

12234.1357421875


0.7133088


# 9.0 Cluster Analyslis

In [35]:
df9 = df6.copy()
df9['cluster'] = labels
X['cluster'] = labels

## 9.2 Cluster Profile

In [36]:
# df9['avg_recency_days'] = df9['avg_recency_days'].apply( lambda x: 1/x if x !=0 else 0)
# df9['recency_days'] = df9['recency_days'].apply( lambda x: 1/x if x !=0 else 0)

In [37]:
# Number of customer
df_cluster = df9[['customer_id', 'cluster']].groupby( 'cluster' ).count().reset_index()
df_cluster['perc_customer'] = 100*( df_cluster['customer_id'] / df_cluster['customer_id'].sum() )

# Avg Gross revenue
df_avg_gross_revenue = df9[['gross_revenue', 'cluster']].groupby( 'cluster' ).mean().reset_index()
df_cluster = pd.merge( df_cluster, df_avg_gross_revenue, how='inner', on='cluster' )

# Avg recency days
df_avg_recency_days = df9[['recency_days', 'cluster']].groupby( 'cluster' ).mean().reset_index()
df_cluster = pd.merge( df_cluster, df_avg_recency_days, how='inner', on='cluster' )

# Avg invoice_no
df_invoice_no = df9[['qtde_invoices', 'cluster']].groupby( 'cluster' ).mean().reset_index()
df_cluster = pd.merge( df_cluster, df_invoice_no, how='inner', on='cluster' )

# Avg Qnt_items
df_items_no = df9[['qtde_items', 'cluster']].groupby( 'cluster' ).mean().reset_index()
df_cluster = pd.merge( df_cluster, df_items_no, how='inner', on='cluster' )

# Avg frequency
df_frequency_no = df9[['frequency', 'cluster']].groupby( 'cluster' ).mean().reset_index()
df_cluster = pd.merge( df_cluster, df_frequency_no, how='inner', on='cluster' )

# Avg returns
df_returns_no = df9[['qtde_returns', 'cluster']].groupby( 'cluster' ).mean().reset_index()
df_cluster = pd.merge( df_cluster, df_returns_no, how='inner', on='cluster' )

In [38]:
df_cluster.sort_values('gross_revenue', ascending=False)

,cluster,customer_id,perc_customer,gross_revenue,recency_days,qtde_invoices,qtde_items,frequency,qtde_returns
7,7,479,11.059801,11225.436701,25.872651,15.181628,6814.043841,0.098949,494.830898
5,5,374,8.635419,2570.682807,41.339572,6.631016,1586.058824,0.107853,22.366310
2,2,445,10.274763,1756.879348,44.058427,5.447191,996.276404,0.075215,20.997753
4,4,573,13.230201,1184.790140,65.394415,3.670157,622.431065,0.161613,10.642234
0,0,433,9.997691,690.503256,91.794457,2.006928,431.879908,0.369581,5.859122
6,6,390,9.004849,555.136436,75.948718,2.835897,262.784615,0.026942,4.056410
1,1,364,8.404526,400.802143,136.392857,1.123626,256.799451,0.998283,3.675824
3,3,381,8.797045,322.561709,129.078740,1.737533,133.217848,0.537492,1.451444
8,8,242,5.587624,290.750744,140.586777,1.099174,163.413223,1.014587,1.838843
9,9,650,15.008081,169.746046,168.864615,1.256923,59.572308,0.874062,1.756923


# 10.0 Deploy

# 11.0 Insert into SQLITE

In [39]:
# Database connection details
host = 'mentoria-db.cj8omc2s6yze.us-east-1.rds.amazonaws.com'
port = '5432'
database = 'postgres'
user = 'postgres'
pwd = '12345678'

# Connection string
endpoint = f'postgresql://{user}:{pwd}@{host}:{port}/{database}'
engine = create_engine(endpoint)

# SQL query to create the table
query_create_table_insiders = """
    CREATE TABLE IF NOT EXISTS insiders (
        customer_id             INTEGER,
        gross_revenue           REAL,
        recency_days            REAL,
        qtde_invoices           INTEGER,
        qtde_items              INTEGER,
        qtde_products           INTEGER,
        avg_ticket              REAL,
        avg_recency_days        REAL,
        frequency               REAL,
        qtde_returns            REAL,
        avg_basket_size         REAL,
        avg_unique_basket_size  REAL,
        revenue_returned        REAL,
        cluster                 INTEGER
    )
"""

# Execute the query as raw SQL
with engine.connect() as conn:
    conn.execute(text(query_create_table_insiders))

# Write DataFrame to the 'insiders' table
df9.to_sql('insiders', con=engine, if_exists='append', index=False)
conn.close()



331